<a href="https://colab.research.google.com/github/yahav1349/Number-of-steps-preditnios/blob/main/internet_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# General

In [4]:
import pandas as pd
import numpy as np

# Modelling
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from zipfile import ZipFile
import random
import warnings
warnings.filterwarnings('ignore')

In [5]:
def norm(x, y, z):
  return np.sqrt(float(x)**2 + float(y)**2 + float(z) **2)

def is_float(string):
    try:
        float(string)
        return True
    except ValueError:
        return False

In [6]:
def split(file_name):
  train_list = []
  test_list = []
  with ZipFile(file_name, "r") as f:
      for name in f.namelist():
        rand = random.uniform(0,1)
        if rand >= 0.8:
          test_list.append(name)
        else:
          train_list.append(name)
  return train_list, test_list

In [228]:
def train_test(curr_list, bad_files):
  mean_norm_list = []
  std_norm_list = []
  max_norm_list = []
  range_list = []
  classification_list = []
  number_of_steps_list = []
  num_rows = []
  for i,name in enumerate(curr_list):
    if name in bad_files:
        data = pd.read_csv(name, skiprows=6)
        head = pd.read_csv(name, nrows=4)
    else:
        data = pd.read_csv(name,skiprows=5)
        head = pd.read_csv(name, nrows=3)
    classification = head.iloc[1,1]
    steps = head.iloc[2,1]
    data['Norm'] = ["" for i in range(len(data))]
    data['Classification'] = [classification for i in range(len(data))]
    for row in range(len(data)):
      x,y,z = data.iloc[row, 1], data.iloc[row, 2], data.iloc[row, 3]
      if  not is_float(x) or not is_float(y) or not is_float(z):
        data.drop(data.index[row])
      else:
        data['Norm'].iloc[row] = norm(x,y,z)
    mean_norm_list.append(np.mean(data['Norm']))
    std_norm_list.append(np.std(data['Norm']))
    max_norm_list.append(np.max(data['Norm']))
    range_list.append(np.max(data['Norm']) - np.min(data['Norm']))
    classification_list.append(data['Classification'][0])
    num_rows.append(len(data))
    number_of_steps_list.append(steps)
  data_final = {
      'Mean Norm': mean_norm_list,
      'Std Norm': std_norm_list,
      'Max Norm': max_norm_list,
      'Range': range_list,
      'Number of Rows': num_rows,
      'Number of Steps': number_of_steps_list,
      'Classification': classification_list
  }
  # Create the DataFrame
  df = pd.DataFrame(data_final)
  return df

In [55]:
def find_bad_lines(df):
  bad_index = []
  for row in range(len(df)):
    x,y,z = df.iloc[row, 1], df.iloc[row, 2], df.iloc[row, 3]
    if '-' in str(x)[1:] or '-' in str(y)[1:] or '-' in str(z)[1:]:
      bad_index.append(row)
  return bad_index

In [67]:
def clean_train(train_df, bad_index):
  train_df = train_df.drop(train_df.index[[i for i in bad_index]])
  # train_df = train_df.iloc[:, :-1]
  train_df = train_df.dropna()
  return train_df

In [72]:
def clean_test(test_df, bad_index):
  test_df = test_df.drop(test_df.index[[i for i in bad_index]])
  # test_df = test_df[['Time [sec]','ACC X','ACC Y', 'ACC Z',	'Norm','Classification']]
  test_df = test_df.dropna()
  return test_df

In [58]:
def null_rows(matrix):
  null_list = []
  null_values = matrix.isnull()
  for column in null_values.columns:
      for index, value in null_values[column].items():
          if value:
              if index not in null_list:
                null_list.append(index)
              print(f"Null value found at index {index} in column {column}")
  return null_list

In [229]:
def return_train_X_y(train):
  train['Number of Steps'] = pd.to_numeric(train['Number of Steps'], errors='coerce')
  X = train.iloc[: , :-1]
  y = train['Classification']
  X['min'] = X['Max Norm'] - X['Range']
  mapping = {'Running': 1, 'Walking': 0,'RUNNING': 1, 'WALKING': 0, }
  y= y.map(mapping)
  return X,y

In [230]:
def return_test_X_y(test):
  test['Number of Steps'] = pd.to_numeric(test['Number of Steps'], errors='coerce')
  null_list = null_rows(test.iloc[: ,:-1])
  test = test.drop(test.index[[i for i in null_list]])
  X = test.iloc[: ,:-1]
  y = test['Classification']
  X['min'] = X['Max Norm'] - X['Range']
  mapping = {'Running': 1, 'Walking': 0,'RUNNING': 1, 'WALKING': 0, }
  y= y.map(mapping)
  return X,y

In [231]:
file_name = 'project_IOT.zip'
bad_files = ["6_run_3_1.csv", "6_run_4_1.csv", "6_walk_5_1.csv", "11_walk_1_1.csv", "11_walk_2_1.csv", "11_walk_3_1.csv", "11_walk_5_1.csv"]
train_list, test_list = split(file_name)
train_df = train_test(train_list, bad_files)
bad_train_index = find_bad_lines(train_df)
train = clean_train(train_df, bad_train_index)
X,y = return_train_X_y(train)

In [218]:
train_df

,Mean Norm,Std Norm,Max Norm,Range,Number of Rows,Number of Steps,Classification
0,11.444922,6.141134,23.795315,22.850169,578,100,Running
1,11.830044,6.559816,30.842323,29.407611,580,100,Running
2,10.109185,2.585022,19.302562,16.641134,957,100,Walking
3,9.997269,2.352843,19.361459,15.743932,972,100,Walking
4,9.892204,2.308173,19.144613,15.775904,945,100,Walking
...,...,...,...,...,...,...,...
174,10.923497,2.076366,18.279182,11.545424,514,80,Walking
175,10.815573,2.210419,19.759722,13.842484,494,75,Walking
176,10.925536,2.238666,20.523560,13.248211,499,75,Walking
177,11.001765,2.384220,20.240121,13.138051,479,77,Walking


In [233]:
test_df = train_test(test_list, bad_files)
bad_test_index = find_bad_lines(test_df)
test = clean_test(test_df, bad_test_index)
X_test, y_test = return_test_X_y(test)

# Running/Walking Classification

In [71]:
!pip install xgboost

**Cross validation for gboost parameters**

In [ ]:
X_g_boost = X.rename(columns={'Time [sec]':'Time'})
from sklearn.model_selection import GridSearchCV
import xgboost as xgb

# Step 1: Define the base model
model = xgb.XGBClassifier()

# Step 2: Define the parameter grid for hyperparameter tuning
param_grid = {
    'learning_rate': [0.01, 0.001],
    'n_estimators': [100, 200],
    'max_depth': [3, 4],
    'subsample': [0.8, 0.9],
    'reg_alpha': [0, 0.1],
    'reg_lambda': [0, 0.1]
}

# Step 3: Perform grid search with cross-validation
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_g_boost, y)

# Step 4: Get the best model and its hyperparameters
best_model = grid_search.best_estimator_
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print("Best Hyperparameters:", best_params)
print("Best Accuracy Score:", best_score)

Best Hyperparameters: {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 100, 'reg_alpha': 0.1, 'reg_lambda': 0.1, 'subsample': 0.8}
Best Accuracy Score: 0.6888820368156463


**Random forest cross validation**

In [238]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

# Step 1: Define the base model
model = RandomForestClassifier()

# Step 2: Define the parameter grid for hyperparameter tuning
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [None, 5],
    'min_samples_split': [2],
    'min_samples_leaf': [1],
    'max_features': ['auto', 'sqrt']
}

# Step 3: Perform grid search with cross-validation
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='accuracy')
grid_search.fit(X, y)

# Step 4: Get the best model and its hyperparameters
best_model = grid_search.best_estimator_
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print("Best Hyperparameters:", best_params)
print("Best Accuracy Score:", best_score)


Best Hyperparameters: {'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
Best Accuracy Score: 0.9288288288288289


**Predictions with the chosen parameters**

In [113]:
train['Classification'].unique()

array(['Running', 'Walking', 'RUNNING', 'WALKING'], dtype=object)

In [241]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
gboost_model = xgb.XGBClassifier(learning_rate= 0.01, max_depth= 3, n_estimators = 100, reg_alpha= 0.1, reg_lambda=0.1, subsample= 0.8)
gboost_model.fit(X, y)
rf_model = RandomForestClassifier(max_features= 'sqrt', min_samples_leaf= 1, min_samples_split=2, n_estimators=200)
rf_model.fit(X, y)

RandomForestClassifier(n_estimators=200)

In [242]:
y_pred_rf = rf_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred_rf)
print("random forest rows Accuracy:", accuracy*100)
# X_test_boost = X_test.rename(columns={'Time [sec]':'Time'})
y_pred_gboost = gboost_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred_gboost)
print("gboost rows Accuracy:", accuracy*100)

random forest rows Accuracy: 97.61904761904762
gboost rows Accuracy: 95.23809523809523


In [131]:
def predict_on_file(bad_files):
  count = 0
  for i,name in enumerate(test_list):
    if 'run' in name:
      classification = 1
    else:
      classification = 0
    test_file = train_test([name], bad_files)
    bad_test_file_index = find_bad_lines(test_file)
    test_file_cleaned =  test_file.drop(test_file.index[[i for i in bad_test_file_index]])
    if 'Time[sec]' in test_file_cleaned.columns:
      test_file_cleaned  = test_file_cleaned .rename(columns={'Time[sec]': 'Time [sec]'})
    for column in test_file_cleaned.columns[1::]:
        test_file_cleaned  = test_file_cleaned .rename(columns={column:column.strip()})
    X_test,_ = return_test_X_y(test_file_cleaned)
    y_pred = rf_model.predict(X_test)
    if round(np.mean(y_pred))==classification:
      count+=1
  print(f'random forest accuracy: {count*100/len(test_list)}%')

In [126]:
def predict_on_gboost(bad_files):
  count = 0
  for i,name in enumerate(test_list):
    if 'run' in name:
      classification = 1
    else:
      classification = 0
    test_file = train_test([name], bad_files)
    bad_test_file_index = find_bad_lines(test_file)
    test_file_cleaned =  test_file.drop(test_file.index[[i for i in bad_test_file_index]])
    if 'Time[sec]' in test_file_cleaned.columns:
      test_file_cleaned  = test_file_cleaned .rename(columns={'Time[sec]': 'Time [sec]'})
    for column in test_file_cleaned.columns[1::]:
        test_file_cleaned  = test_file_cleaned .rename(columns={column:column.strip()})
    X_test,_ = return_test_X_y(test_file_cleaned)
    # X_test_boost = X_test.rename(columns={'Time [sec]':'Time'})
    y_pred = gboost_model.predict(X_test)
    if round(np.mean(y_pred))==classification:
      count+=1
  print(f'gboost accuracy: {count*100/len(test_list)}%')

In [236]:
predict_on_gboost(bad_files)

gboost accuracy: 95.23809523809524%


In [237]:
predict_on_file(bad_files)

random forest accuracy: 97.61904761904762%


# NUmber of Steps Classification

In [202]:
def train_test_steps(curr_list, bad_files):
  mean_norm_list = []
  std_norm_list = []
  max_norm_list = []
  range_list = []
  classification_list = []
  number_of_steps_list = []
  num_rows = []
  for i,name in enumerate(curr_list):
    if name in bad_files:
        data = pd.read_csv(name, skiprows=6)
        head = pd.read_csv(name, nrows=4)
    else:
        data = pd.read_csv(name,skiprows=5)
        head = pd.read_csv(name, nrows=3)
    classification = head.iloc[1,1]
    steps = head.iloc[2,1]
    data['Norm'] = ["" for i in range(len(data))]
    data['Classification'] = [classification for i in range(len(data))]
    for row in range(len(data)):
      x,y,z = data.iloc[row, 1], data.iloc[row, 2], data.iloc[row, 3]
      if  not is_float(x) or not is_float(y) or not is_float(z):
        data.drop(data.index[row])
      else:
        data['Norm'].iloc[row] = norm(x,y,z)
    mean_norm_list.append(np.mean(data['Norm']))
    std_norm_list.append(np.std(data['Norm']))
    max_norm_list.append(np.max(data['Norm']))
    range_list.append(np.max(data['Norm']) - np.min(data['Norm']))
    classification_list.append(data['Classification'][0])
    num_rows.append(len(data))
    number_of_steps_list.append(steps)
  data_final = {
      'Mean Norm': mean_norm_list,
      'Std Norm': std_norm_list,
      'Max Norm': max_norm_list,
      'Range': range_list,
      'Classification': classification_list,
      'Number of Rows': num_rows,
      'Number of Steps': number_of_steps_list
  }
  # Create the DataFrame
  df = pd.DataFrame(data_final)
  return df

In [201]:
file_name = 'project_IOT.zip'
bad_files = ["6_run_3_1.csv", "6_run_4_1.csv", "6_walk_5_1.csv", "11_walk_1_1.csv", "11_walk_2_1.csv", "11_walk_3_1.csv", "11_walk_5_1.csv"]
train_list, test_list = split(file_name)
train_df = train_test(train_list, bad_files)
bad_train_index = find_bad_lines(train_df)
train = clean_train(train_df, bad_train_index)
X_train,y_train = X_y_steps(train)

Index(['Mean Norm', 'Std Norm', 'Max Norm', 'Range', 'Classification',
       'Number of Rows', 'min'],
      dtype='object')


In [206]:
def X_y_steps(df):
  df['Number of Steps'] = pd.to_numeric(df['Number of Steps'], errors='coerce')
  # train['ACC X'] = pd.to_numeric(train['ACC X'], errors='coerce')
  # train['ACC Y'] = pd.to_numeric(train['ACC Y'], errors='coerce')
  # train['ACC Z'] = pd.to_numeric(train['ACC Z'], errors='coerce')
  # train['Norm'] = pd.to_numeric(train['Norm'], errors='coerce')
  mapping = {'Running': 1, 'Walking': 0,'RUNNING': 1, 'WALKING': 0, }
  df['Classification'] = df['Classification'].map(mapping)
  # Separate the target variable 'y' and the feature matrix 'X'
  X = df.iloc[:,:-1]  # Columns 1, 2, 3, 4, and 6
  X['min'] = X['Max Norm'] - X['Range']
  y = df.iloc[:, -1]  # Column 5
  return X,y

In [208]:
test_df = train_test_steps(test_list, bad_files)
bad_test_index = find_bad_lines(test_df)
test = clean_test(test_df, bad_test_index)
X_t, y_t = X_y_steps(test)

In [210]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
gb_regressor = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, random_state=42)

# Fit the regressor to your training data
gb_regressor.fit(X_train, y_train)

# Use the trained regressor to make predictions
y_pred = gb_regressor.predict(X_t)

# Evaluate the model using mean squared error
rmse = np.sqrt(mean_squared_error(y_t, y_pred))
print("Root Mean Squared Error:", rmse)

Root Mean Squared Error: 16.320297885987067
